In [2]:
from toy_classify import *
import ot
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

/users/1/tianx/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## SMOTE

In [4]:
from imblearn.over_sampling import SMOTE

In [5]:
num_sim = 10

#set_seed(666)
# Generate imbalanced data
n_minority= 800
n_majority = 3200
n_val=400
n_test=800
n_train=4000

ratio_list = [x/10 for x in list(range(1,11))]
candidate_m_syn=  [int(n_train*round(x * 0.1, 2)) for x in range(21)]
candidate_alpha_scale= [0.1, 0.2, 0.3,0.4, 0.5,0.6, 0.7,0.8, 0.9]

list_result=[]
ce_smote=[]
ce_origin=[]
list_origin=[]
W_list=[[] for j in range(num_sim)]
X_total, y_total, regions_total = generate_imbalanced_data(
        n_minority + n_val + n_test, 
        n_majority + n_val + n_test, 
        seed=0
    )
y_total = y_total.reshape(-1, 1)    
for j in range(num_sim):
    set_seed(j)
    
    # Separate the minority and majority groups based on region labels.
    # Here, we assume that region label 0 indicates minority and 1 indicates majority.
    minority_idx = np.where(regions_total == 0)[0]
    majority_idx = np.where(regions_total == 1)[0]

    # Shuffle indices separately
    np.random.shuffle(minority_idx)
    np.random.shuffle(majority_idx)

    # For minority:
    min_train_idx = minority_idx[:n_minority]
    min_val_idx   = minority_idx[n_minority : n_minority + n_val]
    min_test_idx  = minority_idx[n_minority + n_val : n_minority + n_val + n_test]

    # For majority:
    maj_train_idx = majority_idx[:n_majority]
    maj_val_idx   = majority_idx[n_majority : n_majority + n_val]
    maj_test_idx  = majority_idx[n_majority + n_val : n_majority + n_val + n_test]

    # Combine directly for each set
    X_train_orig = np.vstack((X_total[min_train_idx], X_total[maj_train_idx]))
    y_train_orig = np.vstack((y_total[min_train_idx], y_total[maj_train_idx]))
    regions_train = np.concatenate((regions_total[min_train_idx], regions_total[maj_train_idx]))

    X_val = np.vstack((X_total[min_val_idx], X_total[maj_val_idx]))
    y_val = np.vstack((y_total[min_val_idx], y_total[maj_val_idx]))
    regions_val = np.concatenate((regions_total[min_val_idx], regions_total[maj_val_idx]))

    X_test = np.vstack((X_total[min_test_idx], X_total[maj_test_idx]))
    y_test = np.vstack((y_total[min_test_idx], y_total[maj_test_idx]))
    regions_test = np.concatenate((regions_total[min_test_idx], regions_total[maj_test_idx]))

    
    XY_train_orig = combine_XY(X_train_orig, y_train_orig)
    
    n_train= XY_train_orig.shape[0]

    origin_model, mse_val_origin, mse_test_origin, y_val_opred, y_test_opred = train_and_evaluate(
    X_train_orig, y_train_orig, X_val, y_val, X_test, y_test, input_dim= X_train_orig.shape[1], batch_size = 1024,patience =20)
    
    list_origin.append([mse_val_origin, mse_test_origin])
    print([mse_val_origin, mse_test_origin])

    all_result=[]
 
        
    for m_syn in candidate_m_syn:   

        result=[]

        for alpha_scale in candidate_alpha_scale:
            # Adjust synthetic data: if alpha_scale < 1, use a fraction; if > 1, replicate data accordingly.
            if m_syn ==0:
                X_train_combined = XY_train_orig[:,:-1]
                y_train_combined = XY_train_orig[:,-1:]    
            else:
                
                resample_dict = {0: int(m_syn*alpha_scale)+ n_minority, 1: int(m_syn*(1-alpha_scale))+ n_majority}

                over_sampler = SMOTE(sampling_strategy=resample_dict, random_state=j)
                X_rs_df, y_rs = over_sampler.fit_resample(XY_train_orig[:, :-1], XY_train_orig[:, -1])
                

                X_train_combined = X_rs_df
                y_train_combined = y_rs


            # Train and evaluate on validation set
            _, mse_val, mse_test, _, _ = train_and_evaluate(
                X_train_combined, y_train_combined, X_val, y_val, X_test, y_test,
                input_dim= X_train_orig.shape[1], batch_size = 1024,patience =20)

            print(f"m_syn={m_syn}, alpha_scale={alpha_scale} -> Validation MSE: {mse_val:.4f} -> Test MSE: {mse_test:.4f}")

            result.append([mse_val, mse_test])


        all_result.append(result)
    
    ax= [all_result[x][:10] for x in range(21)] 
    ax=np.array(ax)
    num_sim, num_m, num_min_ratio = ax.shape

    val_errors = ax[: , :, 0]

    # Find the indices (k, l) that minimize the validation error.
    k, l = np.unravel_index(np.argmin(val_errors), val_errors.shape)

        
    best_m = candidate_m_syn[k]
    best_alpha = candidate_alpha_scale[l]
    
    resample_dict = {0: int(best_m*best_alpha)+ n_minority, 1: int(best_m*(1-best_alpha))+ n_majority}

    over_sampler = SMOTE(sampling_strategy=resample_dict, random_state=0)
    X_rs_df, y_rs = over_sampler.fit_resample(XY_train_orig[:, :-1], XY_train_orig[:, -1])



    X_train_combined = X_rs_df
    y_train_combined = y_rs
    
    ces_one={}


    _, mse_val, mse_test, ypred_val, ypred_test = train_and_evaluate(
        X_train_combined, y_train_combined, X_val, y_val, X_test, y_test,
        input_dim= X_train_orig.shape[1], batch_size = 1024,patience =20)
    ces_one["avg"]=mse_test
    
    _, mse_val, mse_test, ypred_val, ypred_test = train_and_evaluate(
    X_train_combined, y_train_combined, X_val, y_val, X_test[regions_test==1,], y_test[regions_test==1],
    input_dim= X_train_orig.shape[1], batch_size = 1024,patience =20)
    ces_one["major"]=mse_test
    
    _, mse_val, mse_test, ypred_val, ypred_test = train_and_evaluate(
    X_train_combined, y_train_combined, X_val, y_val, X_test[regions_test==0,], y_test[regions_test==0],
    input_dim= X_train_orig.shape[1], batch_size = 1024,patience =20)
    ces_one["minor"]=mse_test
    
    ce_smote.append(ces_one)
    
    ces_one={}
    _, mse_val_origin, mse_test_origin, y_val_opred, y_test_opred = train_and_evaluate(
    X_train_orig, y_train_orig, X_val, y_val, X_test, y_test, input_dim= X_train_orig.shape[1], batch_size = 1024,patience =20)
    ces_one["avg"]=mse_test_origin
    _, mse_val_origin, mse_test_origin, y_val_opred, y_test_opred = train_and_evaluate(
    X_train_orig, y_train_orig, X_val, y_val, X_test[regions_test==1,], y_test[regions_test==1], input_dim= X_train_orig.shape[1], batch_size = 1024,patience =20)
    ces_one["major"]=mse_test_origin
    _, mse_val_origin, mse_test_origin, y_val_opred, y_test_opred = train_and_evaluate(
    X_train_orig, y_train_orig, X_val, y_val, X_test[regions_test==0,], y_test[regions_test==0], input_dim= X_train_orig.shape[1], batch_size = 1024,patience =20)
    ces_one["minor"]=mse_test_origin
    
    ce_origin.append(ces_one)
    list_result.append(all_result)


Validation Accuracy: 0.73625
Test Accuracy: 0.701875
Validation BCE Metric: 0.48582834005355835
Test BCE Metric: 0.6221937537193298
Average Validation Entropy: nan
Average Test Entropy: nan
[0.48582834005355835, 0.6221937537193298]
Validation Accuracy: 0.73625
Test Accuracy: 0.701875
Validation BCE Metric: 0.48582834005355835
Test BCE Metric: 0.6221937537193298
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=0, alpha_scale=0.1 -> Validation MSE: 0.4858 -> Test MSE: 0.6222
Validation Accuracy: 0.73625
Test Accuracy: 0.701875
Validation BCE Metric: 0.48582834005355835
Test BCE Metric: 0.6221937537193298
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=0, alpha_scale=0.2 -> Validation MSE: 0.4858 -> Test MSE: 0.6222
Validation Accuracy: 0.73625
Test Accuracy: 0.701875
Validation BCE Metric: 0.48582834005355835
Test BCE Metric: 0.6221937537193298
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=0, alpha_scale=0.3 -> Validation MSE: 0.4858 -> Te

Validation Accuracy: 0.74875
Test Accuracy: 0.72125
Validation BCE Metric: 0.45337045192718506
Test BCE Metric: 0.6036656498908997
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=1200, alpha_scale=0.4 -> Validation MSE: 0.4534 -> Test MSE: 0.6037
Validation Accuracy: 0.75125
Test Accuracy: 0.718125
Validation BCE Metric: 0.433072954416275
Test BCE Metric: 0.5569245219230652
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=1200, alpha_scale=0.5 -> Validation MSE: 0.4331 -> Test MSE: 0.5569
Early stopping at epoch 92
Validation Accuracy: 0.7425
Test Accuracy: 0.711875
Validation BCE Metric: 0.43713685870170593
Test BCE Metric: 0.5574595928192139
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=1200, alpha_scale=0.6 -> Validation MSE: 0.4371 -> Test MSE: 0.5575
Early stopping at epoch 82
Validation Accuracy: 0.74875
Test Accuracy: 0.7225
Validation BCE Metric: 0.41716331243515015
Test BCE Metric: 0.5212188363075256
Average Validation Entropy: 

Early stopping at epoch 70
Validation Accuracy: 0.765
Test Accuracy: 0.7575
Validation BCE Metric: 0.3915449380874634
Test BCE Metric: 0.5232040882110596
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=2400, alpha_scale=0.7 -> Validation MSE: 0.3915 -> Test MSE: 0.5232
Early stopping at epoch 74
Validation Accuracy: 0.78875
Test Accuracy: 0.75875
Validation BCE Metric: 0.3904901146888733
Test BCE Metric: 0.5384729504585266
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=2400, alpha_scale=0.8 -> Validation MSE: 0.3905 -> Test MSE: 0.5385
Validation Accuracy: 0.81
Test Accuracy: 0.78
Validation BCE Metric: 0.3778121769428253
Test BCE Metric: 0.5772530436515808
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=2400, alpha_scale=0.9 -> Validation MSE: 0.3778 -> Test MSE: 0.5773
Early stopping at epoch 78
Validation Accuracy: 0.72
Test Accuracy: 0.69375
Validation BCE Metric: 0.5620174407958984
Test BCE Metric: 0.7033575177192688
Average Validat

Validation Accuracy: 0.74875
Test Accuracy: 0.726875
Validation BCE Metric: 0.5026397705078125
Test BCE Metric: 0.6955477595329285
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=4000, alpha_scale=0.1 -> Validation MSE: 0.5026 -> Test MSE: 0.6955
Validation Accuracy: 0.77625
Test Accuracy: 0.74875
Validation BCE Metric: 0.45460525155067444
Test BCE Metric: 0.6505611538887024
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=4000, alpha_scale=0.2 -> Validation MSE: 0.4546 -> Test MSE: 0.6506
Early stopping at epoch 65
Validation Accuracy: 0.74625
Test Accuracy: 0.723125
Validation BCE Metric: 0.45868414640426636
Test BCE Metric: 0.582368791103363
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=4000, alpha_scale=0.3 -> Validation MSE: 0.4587 -> Test MSE: 0.5824
Early stopping at epoch 65
Validation Accuracy: 0.765
Test Accuracy: 0.7475
Validation BCE Metric: 0.40712472796440125
Test BCE Metric: 0.5515134930610657
Average Validation Entropy: n

Early stopping at epoch 91
Validation Accuracy: 0.77625
Test Accuracy: 0.76125
Validation BCE Metric: 0.4232906997203827
Test BCE Metric: 0.614573061466217
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=5200, alpha_scale=0.3 -> Validation MSE: 0.4233 -> Test MSE: 0.6146
Early stopping at epoch 80
Validation Accuracy: 0.77125
Test Accuracy: 0.7575
Validation BCE Metric: 0.40423765778541565
Test BCE Metric: 0.6059957146644592
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=5200, alpha_scale=0.4 -> Validation MSE: 0.4042 -> Test MSE: 0.6060
Early stopping at epoch 45
Validation Accuracy: 0.7625
Test Accuracy: 0.733125
Validation BCE Metric: 0.4219856858253479
Test BCE Metric: 0.5400609374046326
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=5200, alpha_scale=0.5 -> Validation MSE: 0.4220 -> Test MSE: 0.5401
Early stopping at epoch 45
Validation Accuracy: 0.775
Test Accuracy: 0.738125
Validation BCE Metric: 0.3995923399925232
Test BCE Metri

...

Early stopping at epoch 58
Validation Accuracy: 0.705
Test Accuracy: 0.713125
Validation BCE Metric: 0.5487169623374939
Test BCE Metric: 0.5181806087493896
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=2400, alpha_scale=0.3 -> Validation MSE: 0.5487 -> Test MSE: 0.5182
Early stopping at epoch 70
Validation Accuracy: 0.745
Test Accuracy: 0.7325
Validation BCE Metric: 0.5388278365135193
Test BCE Metric: 0.49905943870544434
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=2400, alpha_scale=0.4 -> Validation MSE: 0.5388 -> Test MSE: 0.4991
Early stopping at epoch 70
Validation Accuracy: 0.745
Test Accuracy: 0.741875
Validation BCE Metric: 0.514585018157959
Test BCE Metric: 0.48717302083969116
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=2400, alpha_scale=0.5 -> Validation MSE: 0.5146 -> Test MSE: 0.4872
Early stopping at epoch 56
Validation Accuracy: 0.73625
Test Accuracy: 0.73875
Validation BCE Metric: 0.5093743205070496
Test BCE Metric:

Early stopping at epoch 50
Validation Accuracy: 0.73375
Test Accuracy: 0.736875
Validation BCE Metric: 0.49684637784957886
Test BCE Metric: 0.4697688817977905
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=3600, alpha_scale=0.5 -> Validation MSE: 0.4968 -> Test MSE: 0.4698
Early stopping at epoch 44
Validation Accuracy: 0.745
Test Accuracy: 0.749375
Validation BCE Metric: 0.4932771921157837
Test BCE Metric: 0.4765765368938446
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=3600, alpha_scale=0.6 -> Validation MSE: 0.4933 -> Test MSE: 0.4766
Early stopping at epoch 51
Validation Accuracy: 0.7625
Test Accuracy: 0.751875
Validation BCE Metric: 0.48518404364585876
Test BCE Metric: 0.4686530530452728
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=3600, alpha_scale=0.7 -> Validation MSE: 0.4852 -> Test MSE: 0.4687
Early stopping at epoch 54
Validation Accuracy: 0.77125
Test Accuracy: 0.759375
Validation BCE Metric: 0.48005616664886475
Test BCE

Early stopping at epoch 43
Validation Accuracy: 0.76125
Test Accuracy: 0.75125
Validation BCE Metric: 0.48346763849258423
Test BCE Metric: 0.4591725170612335
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=4800, alpha_scale=0.7 -> Validation MSE: 0.4835 -> Test MSE: 0.4592
Early stopping at epoch 42
Validation Accuracy: 0.7425
Test Accuracy: 0.75875
Validation BCE Metric: 0.4813411235809326
Test BCE Metric: 0.45566821098327637
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=4800, alpha_scale=0.8 -> Validation MSE: 0.4813 -> Test MSE: 0.4557
Early stopping at epoch 43
Validation Accuracy: 0.7425
Test Accuracy: 0.743125
Validation BCE Metric: 0.4864935874938965
Test BCE Metric: 0.46329811215400696
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=4800, alpha_scale=0.9 -> Validation MSE: 0.4865 -> Test MSE: 0.4633
Early stopping at epoch 63
Validation Accuracy: 0.7125
Test Accuracy: 0.716875
Validation BCE Metric: 0.6625093817710876
Test BCE M

Early stopping at epoch 51
Validation Accuracy: 0.74875
Test Accuracy: 0.768125
Validation BCE Metric: 0.48288869857788086
Test BCE Metric: 0.44927817583084106
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=6000, alpha_scale=0.9 -> Validation MSE: 0.4829 -> Test MSE: 0.4493
Early stopping at epoch 68
Validation Accuracy: 0.705
Test Accuracy: 0.710625
Validation BCE Metric: 0.6477171182632446
Test BCE Metric: 0.6019400358200073
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=6400, alpha_scale=0.1 -> Validation MSE: 0.6477 -> Test MSE: 0.6019
Early stopping at epoch 54
Validation Accuracy: 0.71625
Test Accuracy: 0.71625
Validation BCE Metric: 0.6039000749588013
Test BCE Metric: 0.5716920495033264
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=6400, alpha_scale=0.2 -> Validation MSE: 0.6039 -> Test MSE: 0.5717
Early stopping at epoch 45
Validation Accuracy: 0.71875
Test Accuracy: 0.719375
Validation BCE Metric: 0.5499460697174072
Test BCE 

Early stopping at epoch 51
Validation Accuracy: 0.71625
Test Accuracy: 0.726875
Validation BCE Metric: 0.6042071580886841
Test BCE Metric: 0.5527991652488708
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=7600, alpha_scale=0.2 -> Validation MSE: 0.6042 -> Test MSE: 0.5528
Early stopping at epoch 36
Validation Accuracy: 0.70875
Test Accuracy: 0.709375
Validation BCE Metric: 0.5610404014587402
Test BCE Metric: 0.5378562211990356
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=7600, alpha_scale=0.3 -> Validation MSE: 0.5610 -> Test MSE: 0.5379
Early stopping at epoch 36
Validation Accuracy: 0.72
Test Accuracy: 0.723125
Validation BCE Metric: 0.5187389254570007
Test BCE Metric: 0.49321699142456055
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=7600, alpha_scale=0.4 -> Validation MSE: 0.5187 -> Test MSE: 0.4932
Early stopping at epoch 46
Validation Accuracy: 0.76125
Test Accuracy: 0.756875
Validation BCE Metric: 0.49365973472595215
Test BCE 

### Summary statistics for Table 1

In [8]:
[np.mean([x['major'] for x in ce_smote]),np.mean([x['minor'] for x in ce_smote]),np.mean([x['avg'] for x in ce_smote])] 

[0.4133892148733139, 0.5344955950975419, 0.4739423990249634]

In [12]:
[np.std([x['major'] for x in ce_smote]),np.std([x['minor'] for x in ce_smote]),np.std([x['avg'] for x in ce_smote])] 

[0.03314269196286765, 0.06065073926969265, 0.036538013496700515]

In [10]:
[np.mean([x['major'] for x in ce_origin]),np.mean([x['minor'] for x in ce_origin]),np.mean([x['avg'] for x in ce_origin])] 

[0.19200099855661393, 0.959399688243866, 0.5757003486156463]

In [11]:
[np.std([x['major'] for x in ce_origin]),np.std([x['minor'] for x in ce_origin]),np.std([x['avg'] for x in ce_origin])] 

[0.019861840831074385, 0.06877081456427804, 0.026359389535778932]